Лаборотарона робота №4
Виконали студенти групи ФБ-05 Алькова Аліна, Супрун Максим

Перша частина містить такі функції:
 - gcdExtended - функція пошуку нсд за розширеним алгоритмом Евкліда (з лаб3)
 - modInverse - функція пошуку оберненого елемента (з лаб3)
 - MillerRabinTest - тест Міллера-Рабіна написаний за алгоритмом поданим у методичці
 - PrimeTest - додатковий тест, що виконується перед тестом Міллера-Рабіна
 - get_prime - генерує просте число

In [142]:
#PART1
import random
def gcdExtended(a, b):
    if a == 0:
        x = 0
        y = 1
        return (abs(b), 0, 1)
    gcd, y, x = gcdExtended(b % a, a) #recursion
    x = x - (b // a) * y
    return (gcd, x, y)
    
def modInverse(a, m):
    gcd, x, y = gcdExtended(a, m)
    if gcd != 1:
        answer = 0 #Inverse doesn't exist
    else:
        answer = ((x % m + m) % m)
    return answer

def MillerRabinTest(p):   #Miller-Rabin Test that helps to check whether number is prime ot not    
    s = 0
    d = p - 1
    while d % 2 == 0:
        s += 1
        d = d // 2
    for trials in range(100):  
        x = random.randrange(1, p+1)
        if gcdExtended(x, p)[0] == 1:  
            if pow(x, d, p) == 1 or pow(x, d, p) == -1 :  
                return True
            else:
                for r in range(1, s - 1):   
                    xr = pow(x, d * (2 ** r), p)
                    if xr == -1:
                        return True  
                    elif xr == 1:
                        return False
                    else:
                        continue
        else:
            return False
    return False

def PrimeTest(num): #additional test
    knownPrimes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101]
    if num in knownPrimes: #if num in list
        return True
    for prime in knownPrimes: # if it is divided by some number from list
        if (num % prime == 0):
            return False
    return MillerRabinTest(num) #go to Miller-Rabin Test


def get_prime():
    while True:
        minr = (2 ** 255) + 1
        maxr = (2 ** 256) - 1
        a = random.randrange(minr,maxr+1)
        if PrimeTest(a): #check if it is ok than return
            return a          

Друга частина містить такі функції:
- get_primepair - генерує пари простих чисел для подальших учасників

In [7]:
#PART2
def get_primepair():
    while True: #p,q,p1,q1
        p = get_prime()
        q = get_prime()
        p1 = get_prime()
        q1 = get_prime()
        pairs = [] #array that will contain pairs
        if (p * q) < (p1 * q1):
            pairs.append(p) #adding each num
            pairs.append(q)
            pairs.append(p1)
            pairs.append(q1)
            return pairs
        
        

p,q,p1,q1=get_primepair()
#print(p,"---",q,"---",p1,"---",q1)

Третя частина містить такі функції:
- get_keypair - генерує ключові пари для RSA
Учасників називатимемо по класиці Алісою та Бобом

In [8]:
#PART3
def get_keypair(p, q): #generate keys due to the rule
    n = p*q
    EulerFunc = (p-1)*(q-1)
    while True:
        e = random.randrange(2, EulerFunc)
        if gcdExtended(e, EulerFunc)[0] == 1:
            d = modInverse(e, EulerFunc)
            return [d, p, q], [n, e] 

a_private, a_public = get_keypair(p, q)
b_private, b_public = get_keypair(p1, q1)
print("---Alice:\n")
print("p: ",p,"\n")
print("q: ",q,"\n")
print("Alice's public key: ",a_public,"\n")
print("Alice's private key: ",a_private[0],"\n")
print("---Bob:\n")
print("p: ",p1,"\n")
print("q: ",q1,"\n")
print("Bob's public key: ",b_public,"\n")
print("Bob's private key: ",b_private[0],"\n")



---Alice:

p:  66940350573416764237533868426136458258565776292343662957536111610763892040179 

q:  59989047430891953102761416942880075497059566499752767885049082593057268089363 

Alice's public key:  [4015687865589233620396443097936281946319589225464015050725107692124014561400627785408109841506920570459826653968766433560727510297543813946197655358515977, 1753944344984322061147632679802677961121946114671474579501661691711458669467694851284524825023479665890085779788455875769650742117461132531209023932618613] 

Alice's private key:  2310417493218980967624041392268888328929314437947963603686121498571471626938640935075228896971761441901403791093674951632274764461924749640546919337234777 

---Bob:

p:  110582499002401853067925970165563507376162886381782711375157884867316635547551 

q:  67694477173663661453187260364098520323641033330507644862420814953188575111263 

Bob's public key:  [748582445452477685666462738914968416928451337424009794251218482699299552433001168665325882177229918397590912

Допоміжні функції, що конверутватимуть текстові повідомлення в потрібну систему числення та навпаки

In [9]:
#PREPARE MESSAGE
import binascii
def convert(message):
    return int(message.encode('utf-8').hex(), base=16)

def convert_totext(message):
    hexmss=hex(message)[2:]
    return binascii.unhexlify(hexmss).decode('utf-8')


text = "Catch me, if u can"
message = convert(text)

Четверта частина містить:
- encrypt - шифрує повідомлення
- decrypt - розшифровує
- sign - генерує підпис
- verify - перевіряє підпис

In [10]:
#PART4
def encrypt(message, public_key):
    return pow(message, public_key[1], public_key[0])
def decrypt(ct, private_key):
    n=private_key[1]*private_key[2]
    return pow(ct, private_key[0],n)
def sign(message,private_key):
    n=private_key[1]*private_key[2]
    return pow(message, private_key[0],n)
def verify(message, sign, public_key):
    testmessage = pow(sign, public_key[1], public_key[0])
    if  testmessage == message:
        print("(!)verification succeeded")
        return True
    else:
        print("(!)verification failed")
        return False
    

Остання частина містить фукції відправки та отримання повідомлення.

In [11]:
#PART5
def send(k, dest_public_key, src_private_key):
    print("--Ready to send message")
    k1 = encrypt(k, dest_public_key) #message
    s = sign(k, src_private_key) # create sign
    s1 = encrypt(s, dest_public_key) #encrypt it
    print("--Sending this ->")
    return [k1, s1]

def recieve(k1s1, dest_private_key, src_public_key):
    print("--Message was recieved")
    k = decrypt(k1s1[0], dest_private_key)
    s = decrypt(k1s1[1], dest_private_key)
    if verify(k, s, src_public_key):
        mess = convert_totext(k)
        print("--Message from receiver ->")
        return mess
    else:
        print("(!)аuthentication failed")
        return None

send=send(message,b_public,a_private)
print(send)
recieve = recieve(send,b_private,a_public)
print(recieve)

--Ready to send message
--Sending this ->
[5383390308571295562903770047128169938687977299663276224210780671005328543879353375257039640902788244757827512469735192455467637235525083319446763475914084, 4729382801463043901252622430869616575452765948357518205031249697686706456742787855215343970390043955842175990609929081340937116426827664070681823363118836]
--Message was recieved
(!)verification succeeded
--Message from receiver ->
Catch me, if u can


In [ ]:
Далі йдуть декілька тестів, що перевірялись за допомогою вказаного сайта у методичних вказівках

In [30]:
def to_hex(number):
    hex_n = hex(number)
    return hex_n[2:]
def from_hex(hex_n):
    numb = int(hex_n, base=16)
    return numb

In [33]:
#TEST1 - ENCRYPTION FUNCTION (ENCRYPT LOCAL, DECRYPT ON SERVER)
server_key_n = '8EFC6EED4D95A432ECB4EAECFEDCB5EB0B5BEBA4EA060B9C8264708D6AB73179'
server_key_n = from_hex(server_key_n)
#print(server_key_n)
server_key_e = '10001'
server_key_e = from_hex(server_key_e)

serv_key_pub = [server_key_n, server_key_e]

test_message = "Glory to Ukraine"
test_message = convert(test_message)

encrypted = to_hex(encrypt(test_message, serv_key_pub))
print('Text is encrypted!')
print("Here is your ciphertext -->  ",encrypted)

Text is encrypted!
Here is your ciphertext -->   1853f208b0159888d65d13c9d3befd0c41d6c13280e7a7dd452e0a06af88020e


In [47]:
#TEST2 - DECRYPTION FUNCTION (DECRYPT LOCAL, ENCRYPT ON SERVER+ENCRYPT LOCAL)
print("Alice's public key: ",a_public,"\n")
my_key_n = to_hex(a_public[0])
#print(my_key_n)
my_key_e = to_hex(a_public[1])
#print(my_key_e)

tmessage = "Catch me if you can"
server_ciphertext = '3A4CE3D550E6A9CEE818DB740A8D05734ADB95E9E08E2BC3C7A824A05A8574BAE1D746C53A48CDB31179950CB555773C007B782CC5090045281FC11D65FF7FE9'

encrypted = to_hex(encrypt(convert(tmessage), a_public))
print(encrypted,"\n")

print("Alice's private key: ",a_private,"\n")
nserver_ciphertext = from_hex(server_ciphertext)
convert_totext(decrypt(nserver_ciphertext, a_private))

Alice's public key:  [4015687865589233620396443097936281946319589225464015050725107692124014561400627785408109841506920570459826653968766433560727510297543813946197655358515977, 1753944344984322061147632679802677961121946114671474579501661691711458669467694851284524825023479665890085779788455875769650742117461132531209023932618613] 

3a4ce3d550e6a9cee818db740a8d05734adb95e9e08e2bc3c7a824a05a8574bae1d746c53a48cdb31179950cb555773c007b782cc5090045281fc11d65ff7fe9 

Alice's private key:  [2310417493218980967624041392268888328929314437947963603686121498571471626938640935075228896971761441901403791093674951632274764461924749640546919337234777, 66940350573416764237533868426136458258565776292343662957536111610763892040179, 59989047430891953102761416942880075497059566499752767885049082593057268089363] 



'Catch me if you can'

In [48]:
#TEST3 - VERIFY FUNCTION (SIGN ON SERVER, VERIFY LOCAL)
server_sign = '64D9B32067F8840071AEF3504F00F3868DABF3D9C2862126041C350025D8C17B'
server_sign = from_hex(server_sign)
server_key_n = '8EFC6EED4D95A432ECB4EAECFEDCB5EB0B5BEBA4EA060B9C8264708D6AB73179'
server_key_n = from_hex(server_key_n)
#print(server_key_n)
server_key_e = '10001'
server_key_e = from_hex(server_key_e)
serv_key_pub = [server_key_n, server_key_e]
mess = 'Hi'
mess = convert(mess)
verify(mess,server_sign,serv_key_pub)

(!)verification succeeded


True

In [53]:
#TEST4 - SIGN FUNCTION (SIGN LOCAL, VERIFY ON SERVER)
print("Alice's private key: ",a_private,"\n")
mess = 'Hi'
mess = convert(mess)
print(to_hex(sign(mess,a_private)),"\n")
print("Alice's public key: ",a_public,"\n")
my_key_n = to_hex(a_public[0])
print(my_key_n)
my_key_e = to_hex(a_public[1])
print(my_key_e)

Alice's private key:  [2310417493218980967624041392268888328929314437947963603686121498571471626938640935075228896971761441901403791093674951632274764461924749640546919337234777, 66940350573416764237533868426136458258565776292343662957536111610763892040179, 59989047430891953102761416942880075497059566499752767885049082593057268089363] 

988e11976ba3f7596c388abb08c03bc1ba0083a82d3ddc2fef3350d18bdd0cd92f99db7ddbc1c390a57a043d5541148d005efe80d25d0a9b6d0ad75b5f6d451 

Alice's public key:  [4015687865589233620396443097936281946319589225464015050725107692124014561400627785408109841506920570459826653968766433560727510297543813946197655358515977, 1753944344984322061147632679802677961121946114671474579501661691711458669467694851284524825023479665890085779788455875769650742117461132531209023932618613] 

4cac45fe98b41925b86a39320da496eca0c33bbc6802c8fd7bd0483c85f7110508bb8f3e512502d3c8adb6da4d2b390bb4da0b3a61ae8dbb546a8130a5e7ab09
217d19f3ad7853a8a64ff6c935e8ce825afba23129f4f9c2c4fa73f51ce66c1c98